In [1]:
!pip3 install gdown
!python3 -m spacy download en
!gdown 1u2pzjXIuhqf6CR2CBI6kqxLtiynLhUEE #gender_name_csv_file
!pip3 install spacytextblob langdetect

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
     |################################| 13.9 MB 3.7 MB/s eta 0:00:01
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1u2pzjXIuhqf6CR2CBI6kqxLtiynLhUEE 



In [56]:
!pip3 install langid

     |################################| 1.9 MB 3.5 MB/s eta 0:00:01
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=580a448980559323da489a15b17f0923ab1fb380ca04e5f73b5f163d3d4491d0
  Stored in directory: /home/vkumar364/.cache/pip/wheels/93/95/a9/c292c9dd8cadb8f2359f1670ff198a40d47167b0be3236e1c8
Successfully built langid


In [3]:
import gzip
import json
import os
import pandas as pd
import langid

def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)        
    return data

In [2]:
!gdown 1V4MLeoEiPQdocCbUHjR_7L9ZmxTufPFe

Downloading...
From: https://drive.google.com/uc?id=1V4MLeoEiPQdocCbUHjR_7L9ZmxTufPFe
To: /home/vkumar364/goodreads_reviews_comics_graphic.json.gz
100%|#########################################| 147M/147M [00:00<00:00, 160MB/s]


In [1]:
def download_and_join_data(ggid,csvname,gendered_df):
#     os.system('gdown '+ ggid)
    data = load_data(csvname)
    df = pd.DataFrame(data)
    final = pd.merge(df, gendered_data, on='user_id', how='left')
    return final[['user_id', 'rating', 'review_text', 'n_votes','n_comments','gender']]

In [4]:
gendered_data = pd.read_csv('goodreads_id_name_gender_new.csv')
gendered_data = gendered_data.drop(columns=['Unnamed: 0','name_x','age','location','friends_count','groups_count'])
gendered_data = gendered_data.rename(columns={'uuid':'user_id'})
gendered_data.head()

,id,username,joined,reviews_count,first_name,user_id,gender
0,50002191,sozerseda,12/2015,184.0,seda,aad6531213b6711ec6bfa5cd3ecac285,female
1,58298653,NaN,08/2016,24.0,olivia,30a37d704d9c28681606a3fbe80ed58f,female
2,36210433,NaN,10/2014,379.0,naomi,f856f6ef2eb4ad702af863175f281f92,female
3,6949799,azima,11/2011,230.0,azima,32d2876b7eb3db5a1ee87f29c01dce4c,female
4,21443818,NaN,NaN,NaN,nicholas,2ace7a7c6edcdd8ac788fe4fd552b7ca,male


In [7]:
selected = download_and_join_data('1THnnmE4XSCvMkGOsqapQr2cJI5amKA6X','goodreads_reviews_comics_graphic.json.gz',gendered_data)

In [8]:
selected.head()

,user_id,rating,review_text,n_votes,n_comments,gender
0,dc3763cdb9b2cae805882878eebb6a32,3,Sherlock Holmes and the Vampires of London \n ...,0,0,male
1,bafc2d50014200cda7cb2b6acd60cd73,4,"I've never really liked Spider-Man. I am, howe...",0,0,female
2,bafc2d50014200cda7cb2b6acd60cd73,4,"A very quick introduction, this is coming out ...",0,0,female
3,bafc2d50014200cda7cb2b6acd60cd73,5,I've been waiting so long for this. I first st...,1,0,female
4,bafc2d50014200cda7cb2b6acd60cd73,4,The only thing more entertaining than this boo...,0,0,female


In [9]:
selected_final = selected[selected['gender']!='UNK']

In [10]:
selected_final['gender'].value_counts()

female    273097
male      181190
Name: gender, dtype: int64

In [11]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
!pip install contractions

[nltk_data] Downloading package stopwords to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [13]:
import contractions 
import re
from nltk.corpus import stopwords
import numpy as np

In [14]:
expand_contractions = True
remove_stopwords = False
format_text = True
tokenize = False
lemmatize = False

def clean_text(text):
    '''Text Preprocessing '''
    
    # Convert words to lower case
    text = text.lower()
    split_text = text.split()
    
    # Expand contractions
    if expand_contractions:
        expanded_words = []   
        for word in split_text:
            expanded_words.append(contractions.fix(word))
        text = " ".join(expanded_words)

    # Format words and remove unwanted characters
    if format_text:
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text) 
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)
    
    # remove stopwords
    if remove_stopwords:
        split_text = text.split()
        stops = set(stopwords.words("english"))
        word = [w for w in split_text if not w in stops]
        text = " ".join(word)

    # Tokenize each word
    if tokenize:
        text =  nltk.WordPunctTokenizer().tokenize(text)
    
    # Lemmatize each token
    if lemmatize:
        lemm = nltk.stem.WordNetLemmatizer()
        text = list(map(lambda word:list(map(lemm.lemmatize, word)), text))
    
    return text

In [15]:
selected['review_length'] = selected['review_text'].apply(lambda x: len(x.split(' ')))

In [16]:
def get_subsampled_gendered(selected_final,lower,upper):
    selected_final_male = selected_final[selected_final['gender']=='male']
    selected_final_female = selected_final[selected_final['gender']=='female']
    selected_final_male_rev = selected_final_male[selected_final_male['review_length']>lower]
    selected_final_male_rev = selected_final_male_rev[selected_final_male_rev['review_length']<=upper]
    selected_final_female_rev = selected_final_female[selected_final_female['review_length']>lower]
    selected_final_female_rev = selected_final_female_rev[selected_final_female_rev['review_length']<=upper]
    #subsampling here
    min_select = min(list(selected_final_female_rev['rating'].value_counts()) + list(selected_final_male_rev['rating'].value_counts()))
    #sampling down the total data - for males
    x = selected_final_male_rev[selected_final_male_rev['rating']==1].sample(min_select)
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==2].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==3].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==4].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==5].sample(min_select))
    #sampling down the total data - for females
    y = selected_final_female_rev[selected_final_female_rev['rating']==1].sample(min_select)
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==2].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==3].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==4].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==5].sample(min_select))
    return x,y
    

In [17]:
male_rev,female_rev = get_subsampled_gendered(selected,5,500)

In [21]:
female_rev['rating'].value_counts()

1    2920
2    2920
3    2920
4    2920
5    2920
Name: rating, dtype: int64

In [23]:
def clean_en_sentiment(df):
    df['review_text_cleaned'] = df['review_text'].apply(clean_text)
    df['review_lang'] = df['review_text'].apply(lambda x: langid.classify(x)[0])
    df = df[df['review_lang'] == 'en']
    df['sentiment_score'] = df['review_text'].apply(lambda x: nlp(x)._.polarity)
    return df 

In [24]:
male_rev_cleaned = clean_en_sentiment(male_rev)
female_rev_cleaned = clean_en_sentiment(female_rev)

/tmp/ipykernel_27074/1754966298.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_score'] = df['review_text'].apply(lambda x: nlp(x)._.polarity)


In [25]:
female_rev_cleaned = female_rev_cleaned.reset_index()
male_rev_cleaned = male_rev_cleaned.reset_index()

In [26]:
male_rev_cleaned = male_rev_cleaned[['rating','n_votes', 'n_comments', 'gender','review_length','sentiment_score']]
female_rev_cleaned = female_rev_cleaned[['rating','n_votes', 'n_comments', 'gender','review_length','sentiment_score']]

In [27]:
male_rev_cleaned

,rating,n_votes,n_comments,gender,review_length,sentiment_score
0,1,1,0,male,113,0.100000
1,1,0,0,male,19,0.700000
2,1,0,0,male,51,0.061905
3,1,1,2,male,285,0.067744
4,1,1,0,male,11,0.214286
...,...,...,...,...,...,...
13123,5,0,0,male,163,0.062517
13124,5,11,0,male,72,0.543304
13125,5,1,0,male,41,0.357500
13126,5,3,0,male,182,0.269844


In [28]:
male_rev_cleaned['gender'] = 0
female_rev_cleaned['gender'] = 1

In [29]:
combined_data = male_rev_cleaned.append(female_rev_cleaned)

In [36]:
combined_data

,rating,n_votes,n_comments,gender,review_length,sentiment_score
0,1,1,0,0,113,0.100000
1,1,0,0,0,19,0.700000
2,1,0,0,0,51,0.061905
3,1,1,2,0,285,0.067744
4,1,1,0,0,11,0.214286
...,...,...,...,...,...,...
13529,5,0,0,1,31,0.135000
13530,5,0,0,1,65,0.198333
13531,5,0,0,1,38,0.025000
13532,5,0,0,1,35,0.202083


In [37]:
final_g = combined_data[['n_votes', 'n_comments', 'gender', 'review_length','sentiment_score']]
final_w = combined_data[['n_votes', 'n_comments', 'review_length','sentiment_score']]
pred = combined_data['rating']

In [38]:
final_g.head()

,n_votes,n_comments,gender,review_length,sentiment_score
0,1,0,0,113,0.100000
1,0,0,0,19,0.700000
2,0,0,0,51,0.061905
3,1,2,0,285,0.067744
4,1,0,0,11,0.214286


In [39]:
import statsmodels.api as sm
model_g = sm.OLS(pred, sm.add_constant(final_g)).fit()
model_w = sm.OLS(pred, sm.add_constant(final_w)).fit()

/nethome/vkumar364/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [40]:
model_g.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.180
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     1173.
Date:                Tue, 19 Apr 2022   Prob (F-statistic):               0.00
Time:                        23:39:12   Log-Likelihood:                -44464.
No. Observations:               26662   AIC:                         8.894e+04
Df Residuals:                   26656   BIC:                         8.899e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.6400      0.015    179.118      0.000       2.611       2.669
n_votes             0.0024      0.002      1.149      0.251      -0.002       0.006
n_comments         -0.0297      0.006     -5.235      0.000      -0.041      -0.019
gender             -0.0551      0.016     -3.492      0.000      -0.086      -0.024
review_length   -5.028e-05   8.61e-05     -0.584      0.559      -0.000       0.000
sentiment_score     2.5431      0.034     75.881      0.000       2.477       2.609
==============================================================================
Omnibus:                     2326.046   Durbin-Watson:                   0.357
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              733.804
Skew:                           0.058   Prob(JB):                    4.53e-160
Kurtosis:                       2.196   Cond. No.                         562.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
sm.stats.anova_lm(model_w, model_g)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,26657.0,43868.169320,0.0,NaN,NaN,NaN
1,26656.0,43848.105595,1.0,20.063725,12.197076,0.000479
